In [3]:
import pandas as pd
import numpy as np
import random

import openai
import os
from tqdm import tqdm
import time

import sys

sys.path.insert(0, '..')

In [4]:
# TURBO COST
TURBO_COST = 0.00200 / 1000

In [5]:
# Initialize the API client
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [6]:
df = pd.read_csv('../data/seed/youtube/seed_videos.csv')

In [56]:
### BASE PROMPT
# Labels:

# - Blockchain: Cryptocurrency, NFTs, or other Web3 investments
# - Traditional: Stocks, Bonds, Real Estate, Commodities, or other traditional investments
# - Mixed: Videos that recommend or educate about both blockchain and traditional investments
# - None: Videos that do not recommend or educate about either blockchain or traditional investments

# Instructions:

# - If a video snippet mentions keywords related to blockchain or traditional investments but does not recommend or educate about the label, it will be classified as None.
# - Please return predictions in the format" {Label} : {50 word or shorter rationale}"""},
#         {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

# {Label} : {50-word or shorter rationale}

# Please provide me with the YouTube video snippet you would like me to classify.

In [57]:
### Deal with failure to classify retirement accounts
# Labels:

# - Blockchain: Cryptocurrency, NFTs, or other Web3 investments
# - Traditional: Stocks, Bonds, Real Estate, Commodities, Retirement Accounts
# - Mixed: Videos that recommend or educate about both blockchain and traditional investments
# - None: Videos that do not recommend or educate about either blockchain or traditional investments

# Instructions:

# - If a video snippet mentions keywords related to blockchain or traditional investments but does not recommend or educate about the label, it will be classified as None.
# - Please return predictions in the format" {Label} : {50 word or shorter rationale}"""},
#         {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

# {Label} : {50-word or shorter rationale}

# Please provide me with the YouTube video snippet you would like me to classify.

In [ ]:
### Better accuracy failed other things
# Labels:

# Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
# Blockchain: Videos that recommend or educate about cryptocurrency, NFTs, or other Web3 investments or keywords related to them.
# Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
# Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

# Instructions:
# If a video snippet mentions any of the keywords related to traditional or blockchain investments, it will be classified accordingly. The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.

In [ ]:
### High accuracy, great precision recall on traditional and blockchain, really bad at None/Unrelated
# Labels:

# Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
# Blockchain: Videos that recommend or educate about cryptocurrency, NFTs, or other Web3 investments or keywords related to them.
# Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
# Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

# Instructions:
#  - If a video snippet mentions any of the keywords related to traditional or blockchain investments, it will be classified accordingly. The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
#  - Please return predictions in the format" {Label} : {20 word or shorter rationale}

In [24]:
def create_messages(snippet):
    messages = [
        {"role": "system", "content" : "You are a classifier that determines if a YouTube video snippet falls under a label. A snippet is a concatenation of the video title, summarized transcript, and video tags. The labels and additional instructions will be included in the first user message."},
        {"role": "user", "content" : """Labels:

Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
Blockchain: Videos that recommend or educate about cryptocurrency (BTC, ETH, etc.), NFTs, or other Web3 investments or keywords related to them.
Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

Instructions:
- The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
- If talks about making money from jobs, side hustles, or other alternative assets (cars, watches, artificial intelligence, trading cards, art, etc), they are Unrelated.
- If a video is only discussing the asset(s) negatively, it should be classified as Unrelated.
- Please return predictions in the format" {Label} : {20 word or shorter rationale}"""},
        {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

{Label} : {20-word or shorter rationale}

Please provide me with the YouTube video snippet you would like me to classify."""},
    ]
    
    snippet_message = {"role" : "user", "content" : snippet.replace("\n", " ").replace("  ", " ")}
    
    messages.append(snippet_message)
    
    return messages

In [25]:
def classify(messages):
    chatCompletion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0.25
    )
    return chatCompletion

In [26]:
df['messages'] = df['snippet'].apply(create_messages)

In [80]:
completions = []
completion_df = []


pbar = tqdm(tough_traditional.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    
    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(10)

Processing Understand the Blockchain in Two Minutes: : 10it [02:04, 12.47s/it]                                   


In [81]:
predictions = pd.read_csv('seed_predictions.csv')

In [83]:
for reason in predictions['reason']:
    print(reason + '\n')

This video snippet recommends and educates about dividend stocks, which are traditional investments. The speaker explains why companies pay dividends and shares their experience with investing in dividend stocks. There is no mention of blockchain or Web3 investments.

This video snippet recommends and educates about investing in dividend ETFs, which is a traditional investment. The speaker discusses the benefits of investing in SCHD, a dividend ETF, and highlights its consistent gains and cash flow. The video also provides an example of how investing in SCHD can result in significant returns over a 10, 20, or 30-year period. The speaker emphasizes the importance of investing in a fund like SCHD to receive consistent cash flow payments and grow your money over time.

This video snippet discusses Bank CDs and Brokered CDs, which are traditional investments related to savings and finance. The speaker explains the differences between the two and the pros and cons of each option. The video 

In [84]:
sum(predictions['total_tokens']) * TURBO_COST

0.014648

In [85]:
predictions['prediction']

0    Traditional
1    Traditional
2    Traditional
3    Traditional
4    Traditional
5    Traditional
6    Traditional
7    Traditional
8     Blockchain
9     Blockchain
Name: prediction, dtype: object

In [51]:
print(tough_traditional['messages'][34][3]['content'])

Should You Invest in I Bonds?. happy New Year well I know that the. 2023 kicks off are I bonds but I know. investment opportunity even though. inflation has come down a little bit the. interest rate on these bonds is actually. that for six months after purchase now. once youve had that six months then it. to come down from that almost seven. percent that were seeing right now why. going to be coming down but for many. investors who want to keep money in cash. in a safe place its a great way to have. that money work for them but before you. go ahead and go to the treasury direct. website I want you to know that there. are three things that you need to. those bonds by April 30th 2023 to lock. secondly you can put in ten thousand. dollars per person per calendar year if. youre getting a federal tax refund you. five thousand dollars to get you to a. need to hold this money for one year so. if it is cash that you need for a house. down payment in the next three months. not where you want to

In [97]:
print(df['messages'][0][1]['content'])

Labels:

Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
Blockchain: Videos that recommend or educate about cryptocurrency, NFTs, or other Web3 investments or keywords related to them.
Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

Instructions:
 - If a video snippet mentions any of the keywords related to traditional or blockchain investments, it will be classified accordingly. The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
 - Please return predictions in the format" {Label} : {20 word or shorter rationale}


In [98]:
completions = []
completion_df = []

pbar = tqdm(df.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    completion['message'] = entry['messages']
    completion['title'] = entry['title']
    completion['transcript'] = entry['cleaned_transcript']
    completion['snippet'] = entry['snippet']
    completion['link'] = entry['link']

    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(5)

Processing NFTs, Explained: : 100it [12:58,  7.79s/it]                                                                                    


In [138]:
predictions = pd.read_csv('seed_predictions.csv')

In [150]:
preds = list(predictions['prediction'].apply(lambda x: 'None' if x == 'Unrelated' else x))
actual = list(df['label'])
preds = ['Blockchain' if pred == 'Label' else pred for pred in preds]
sum(predictions['total_tokens']) * TURBO_COST

In [153]:
from sklearn.metrics import accuracy_score, classification_report

In [154]:
accuracy_score(actual, preds)

0.83

In [155]:
print(classification_report(actual, preds))

              precision    recall  f1-score   support

  Blockchain       0.93      0.95      0.94        40
       Mixed       0.33      1.00      0.50         5
        None       1.00      0.07      0.12        15
 Traditional       0.91      0.97      0.94        40

    accuracy                           0.83       100
   macro avg       0.79      0.75      0.63       100
weighted avg       0.90      0.83      0.79       100



In [157]:
predictions

,role,content,prediction,reason,created,id,model,object,prompt_tokens,completion_tokens,total_tokens
0,assistant,Traditional: This video snippet discusses inve...,Traditional,This video snippet discusses investing in stoc...,1678236142,chatcmpl-6rcAoO02J19m8FvhFf5NAU2eXNm6l,gpt-3.5-turbo-0301,chat.completion,644,55,699
1,assistant,Traditional: The video snippet provides advice...,Traditional,The video snippet provides advice on how to in...,1678236148,chatcmpl-6rcAujzkpYZzG9ddtrZ2v6W0eABLE,gpt-3.5-turbo-0301,chat.completion,643,47,690
2,assistant,Traditional: The video snippet provides an exp...,Traditional,The video snippet provides an explanation of t...,1678236156,chatcmpl-6rcB2ulDNEFlJpMyzSDfK2ZUWM7US,gpt-3.5-turbo-0301,chat.completion,646,76,722
3,assistant,Traditional: The video snippet provides educat...,Traditional,The video snippet provides education about a r...,1678236163,chatcmpl-6rcB9NPS8kF70STq6h80s8reARJtX,gpt-3.5-turbo-0301,chat.completion,672,62,734
4,assistant,Traditional: This video snippet discusses reti...,Traditional,This video snippet discusses retirement accoun...,1678236171,chatcmpl-6rcBHhZGc2gQAjq5WmBm6jLiV9eCw,gpt-3.5-turbo-0301,chat.completion,695,77,772
5,assistant,Traditional: The video snippet is discussing i...,Traditional,"The video snippet is discussing index funds, w...",1678236178,chatcmpl-6rcBOnSJiJdiIImYxAZYWang3V0H7,gpt-3.5-turbo-0301,chat.completion,692,49,741
6,assistant,Traditional: The video snippet explains the co...,Traditional,The video snippet explains the concept of inde...,1678236186,chatcmpl-6rcBWovkKoF5p5aW9xEzl20MeYVIN,gpt-3.5-turbo-0301,chat.completion,689,62,751
7,assistant,Traditional: This video snippet discusses inde...,Traditional,This video snippet discusses index fund invest...,1678236193,chatcmpl-6rcBdiWY1XYKR8LpsyrgwwCk0tLgT,gpt-3.5-turbo-0301,chat.completion,656,64,720
8,assistant,Traditional: This video snippet discusses the ...,Traditional,This video snippet discusses the basics of bon...,1678236200,chatcmpl-6rcBk0PH4b7ngVpXe0tKI9WKI3QY8,gpt-3.5-turbo-0301,chat.completion,657,73,730
9,assistant,Traditional: This video snippet explains what ...,Traditional,This video snippet explains what bonds are and...,1678236208,chatcmpl-6rcBsk9iMnnUSwjvOh2Za19sgNlGD,gpt-3.5-turbo-0301,chat.completion,653,65,718


In [242]:
df['label'][70:81]

70    Blockchain
71    Blockchain
72    Blockchain
73    Blockchain
74    Blockchain
75    Blockchain
76         Mixed
77         Mixed
78         Mixed
79         Mixed
80         Mixed
Name: label, dtype: object

In [171]:
print(df.iloc[23]['snippet'])

Economic Update on Inflation - PPI Report and Jobless Claims a Problem For the Fed. so obviously thats good for the labor markets but thats bad for inflation and that is what the Federal Reserve is looking at so I do want to mention something about jobless claims this rule it depends on your States but any Severance that people receive that may delay or reduce eligibility your for unemployment benefits therefore many of the recently announced layoffs they may not be appearing in the numbers right now but it is estimated to be as high as 50 000 a week but even if you factor that in thats going to get you close to 250 000 thats nowhere close to four hundred thousand and then you have the retail sales reports January had the largest monthly Rebound in two years retail sales grew in almost every category were talking cars Furniture appliances restaurants travel Leisure Electronics general merchandise all up across the board so this is great for the economy but this is not good for inflatio

In [173]:
predictions.iloc[23]['reason']

'The video snippet discusses both traditional and blockchain investments. It mentions inflation, jobless claims, and retail sales, which are related to traditional investments. It also mentions Tesla, Shiba, and CPI, which are related to blockchain investments. The snippet also touches on the impact of interest rates on corporate profits and margins, which is relevant to both traditional and blockchain investments.'

In [226]:
### This should not be Mixed, it should be traditional. It just has video tags that mentions crypto

In [231]:
print(df.iloc[54]['messages'][3]['content'])

How To Make Money With Coinbase in 2023 (Beginners Guide). so in this video were going to talk. dollars per day using coinbase. money online videos dont forget to drop. this video youtube is going to show you. investments you can buy on coinbase. actually is if you dont know what it is. making money on coinbase today so. you can get five dollars in free crypto. an easy free five dollars right there. for your crypto youre going to be. now if you dont know what youre doing. this video is to start making a hundred. dollars per day and i want to make this. dollars a day consistently. start to earning some free money and. initial amount of investment to start so. many people dont really know the full. dollars to invest. going to be right i dont know if its. going to continue to drop i dont know. started with dollar cost averaging into. week or month and know how much money. initial investment you really do want to. that ive invested and over time in a. had invested into bitcoin just a year.

In [180]:
predictions.iloc[54]['reason']

'This video snippet mentions both traditional and blockchain investments. The speaker talks about making money through Coinbase, a cryptocurrency exchange, and mentions investing in traditional investments like stocks and bonds. The snippet also includes keywords related to both traditional and blockchain investments, such as "retirement accounts," "crypto," and "investment portfolio."'

In [227]:
### This should be blockchain, not mixed. Shift focus to when it's mainly talking about bitcoin

In [236]:
print(df.iloc[65]['messages'][3]['content'])

SHIBA INU ABOUT TO PUMP, SHIBARUIM ON SCHEDULE? Why I AM BUYING More Shiba Inu! #DOGE #XRP. right now its you know its running about 25 000  so that is pretty peachy King guys so were gonna  . be talking about uh some interesting stuff thats  going to be happening should you be buying should  . the Nang Vietnam guys it is looking pretty good  guys so dont forget weve got I dont know if  . holding holding their Bitcoin thats what that  means right now so lets talk about xrp uh for  . going to be successful with their appending  court case thats going on right now you know  . is going to be doing some making some movie move  move its coming pretty soon guys so uh you know  . not liked xrp because of this pending uh court  case thats happening but uh you know all all  . and you and YouTube black youre gonna love it and  you know whats going to happen is if more people  . shirt right so thats one of the reasons why Im  a bit salty about xrp but Im going to be sweet  . one now dont forget w

In [182]:
predictions.iloc[65]['reason']

"The video snippet mentions both traditional investments (BTC) and blockchain investments (SHIBA INU, SHIBARUIM, DOGE, XRP). The speaker discusses the potential for SHIBA INU to pump and mentions the upcoming shabir event, while also mentioning the price of BTC and the success of XRP's court case."

In [ ]:
### BTC is not an investment, maybe provide examples of bitcoin ticker symbols

In [232]:
print(df.iloc[81]['messages'][3]['content'])

How Mark Cuban Turned His First Million Dollars Into A Billion | GQ Sports. made saved and spent my first million. garbage bags why dont you sell them in. every time you need garbage bags all you. Pitt full time thats where things. time I started a company called micro. sold micro solutions for six million. brought in 2 million to me I live like a. stock market I was worth 20 plus million. dollars life was good then one of my. was like you know theres this whole new. thing called the internet theres got to. thing you know video comes along and we. turned it into a company called. what whats the stock going to open at. Im worth 300 million dollars plus. we sold the company to Yahoo for stock. the whole thing could crash dont be. dollars and it was the most insane thing. saved some money we dont quite have a. first thing I bought Gulfstream G5 now I. Ill buy biggest online purchase in the. time and the one thing I can do is save. 285 million dollars for the Dallas. get a big office I did

In [188]:
print(predictions.iloc[81]['reason'])

The video snippet mentions both traditional and blockchain investments. Mark Cuban talks about his experience investing in traditional stocks and real estate, as well as his success with internet and video-related companies, which could be considered blockchain investments.


In [ ]:
### Speculating on bitcoin.

In [189]:
print(df.iloc[82]['snippet'])

"I Got Rich When I Understood This" | Jeff Bezos. someone who didnt already have a good.
and he told me dave i want you to invest.
gonna start a bookstore in my garage.
okay im gonna put it online people buy.
the world ill do over 100 billion.
a hundred billion dollars.
years old that hes gonna be the richest.
man in the world because he has a garage.
going at the right way at the perfect.
time im gonna make as much as i can as.
theres no way this is gonna be a big.
i took the less safe path.
and she wanted me to follow my passion.
thats the kind of thing that really.
gifts in life and you want to take.
hard work and your choices.
so you know you may be the kinds of.
gifts you get you know you might be.
really good at math it might be really.
easy for you thats a kind of gift.
thats a choice you cant really be.
you choose to work hard.
um you choose to do hard things.
those are choices that you can be proud.
proud of your gifts.
proud of your gifts.
proud of your gifts.
because theyre 

In [190]:
print(predictions.iloc[82]['reason'])

The video snippet does not mention any keywords related to traditional or blockchain investments. It is a motivational speech by Jeff Bezos about the importance of hard work, choices, and taking pride in one's gifts.


In [233]:
print(df.iloc[83]['messages'][3]['content'])

How TikTok Could Become a U.S. Company | WSJ. The first concern is that the Chinese government could order TikTok to hand over data about individual American users. The second big thing is the video recommendation algorithm, and the idea is that the Chinese government could order TikTok to control what videos Americans could watch. As for TikToks recommendation algorithm, that was created by its Chinese parent company ByteDance. One option is a divestiture order or a forced sale, which could cut ties to Chinese ownership. The Chinese government would theoretically have less power over TikTok, to force TikTok to do something like spy or control what videos Americans watch. [Narrator] A divestiture order is what happened to the dating app Grindr in 2019, after US officials determined that the apps personal data could potentially be exploited for blackmail, since the majority stakeholder was a Chinese company. Then the buyer and seller might have to go back to CFIUS in order to get the sa

In [192]:
print(predictions.iloc[83]['reason'])

The video snippet discusses both traditional (divestiture order) and blockchain (ByteDance, TikTok) investments. The snippet also mentions the potential challenges facing a forced sale of TikTok to a US buyer, including the Chinese government's power to restrict the exports of technology, such as TikTok's video recommendation algorithm.


In [234]:
print(df.iloc[84]['messages'][3]['content'])

The #1 Wealth Killer No One Talks About. its going to be the cost that actually. nice car or a nice ride and I think a. lot of people buy nicer of a car than. 728 dollars per year so let that sink in. a little bit thats a lot of money so. expenses of owning a car and Im going. save money in every single one of these. economic car its not crazy expensive. cost of ownership every single year is. costing you a lot more money than you. might think all right so lets actually. invest that money into a car the. invest that money into a car the. cars are going to have different curves. cars if you want to save a lot of money. on depreciation you want to buy a car. one of those cars youre going to save a. lot of money on depreciation all right. cars one way to reduce this cost is to. car than you actually need thats. costing you fees every single year in. and finance a car not many people will. thousand dollars down on a car and then. five years so assuming you have a car. providers the thing w

In [194]:
print(predictions.iloc[84]['reason'])

The video snippet discusses the cost of owning a car and how it can impact one's wealth. It provides tips on how to save money on car expenses, which is related to traditional investments.


In [ ]:
# Arguably, 85 is mislabelled by me hahaha...

In [238]:
print(df.iloc[85]['messages'][3]['content'])

5 Money Mistakes the Super Rich Avoid. money mistakes that the ultra wealthy. mistakes what youre going to do is. probably save thousands of dollars down. buying two years worth of toothpaste. thats an immediate return on your money. Investments that youre better off when. theyre on a huge sale on Amazon buy. here which is if youre going to buy. that you know that youre going to need. about a 20 return on your money which is. time and youre saving gas and money the. worst money mistake here is to buy. their money and its one of the ways. zero six percent of his actual net worth. watches in the thousand dollar price. buy on Amazon for 46 dollars so is it. watch worth thousands of dollars well. ultra wealthy like to spend a little bit. thousand dollars gone of your net pay. they spend their money on so lets. money would come along the way and you. bit more money on some of the things I. Sushi so I really spend a lot more money. in my videos before all right mistake. long-term investors a

In [203]:
print(predictions.iloc[85]['reason'])

The video snippet talks about investments, savings, and long-term returns. It mentions stocks, mutual funds, and index funds, which are all traditional investments. The focus is on avoiding money mistakes and making smart investment decisions, which is a common theme in traditional investment education.


In [240]:
print(df.iloc[86]['messages'][3]['content'])

The 7 BEST Side Hustles That Make $100+ Per day. hundred dollars a day with a side hustle. are my top free side hustles and income. gonna start until you hit the like. selling for cash places like ebay. theres a limit to how much youre able. called free time. your income or anything like youre not. extra 100 to 600 a month working a few. hes right theres a lot of great stuff. yeah its free seriously just look at. want a zero dollar side hustle that you. nobody wants what a waste of time and. working refrigerator for free no joke. not like people are gonna be giving away. dont you hate when youre so busy. free i dont even know whats going on. youre looking for a side hustle. money part-time work is usually the. work and if youre looking to earn extra. to do it even if you work a full-time. which you sign up for a new credit card. were gonna cost free and to get a credit. technically spend money you know what uh. youre right this is gonna require that. you spend money but most likely you

In [205]:
print(predictions.iloc[86]['reason'])

The video snippet mentions both traditional and blockchain investments. It talks about side hustles that make money through selling items on eBay (traditional) and also mentions Robinhood (blockchain) as a platform for stock trading.


In [241]:
print(df.iloc[87]['messages'][3]['content'])

7 Side Hustles That Will Make Money ASAP (2022). hustle to make some extra money and if. youre watching this video right now. side hustle ideas over the past 10 years. of time so thats why in this video i. hustles that ive seen work really well. this video with that ive actually been. or maybe they want to learn a new skill. likely than not youre going to have. i started making videos on youtube about. able to do it without making money. channel i was able to start slowly. thousand likes so go ahead and tap the. i only want to make that video because i. can range from things like video editing. scenes like this is a great way to go. money today by looking to work on. id pay good money for any freelancers. hustle for you then actually make sure. things that content creators might need. still make money from it from time to. sneakers like retro air jordans nikes. my money or i could wear the same shoes. like i did for some pairs but another. like an investment and an easy way to. because

In [207]:
print(predictions.iloc[87]['reason'])

This video snippet mentions both traditional and blockchain investments. The speaker discusses traditional side hustles such as freelance work and taking advantage of bank account bonuses, as well as blockchain-related side hustles such as buying and selling sneakers on apps like StockX.


In [243]:
print(df.iloc[88]['messages'][3]['content'])

I Found The EASIEST Way to Make $1000 with AI. way is to use the AI tools to create. tools themselves and they dont need to. little effort so in this video Im going. top secret AI tool that you do not want. done a lot of videos about affiliate. were Affiliates we dont have to create. sign up as an affiliate you can create. people dont succeed with affiliate. of their way and actually want to use. your affiliate link Im also going to. so a lot of people that try affiliate. grab their affiliate link theyll post. want people to use your link you need to. found an awesome tool thats going to. create a very simple step-by-step guide. this tool and thats what were going to. here click on try for free Im going to. through the steps its going to create. got created a step-by-step guide with. create some kind of step-by-step guide. affiliate Link in here so now weve got. actually going to get people coming to. got no money and you actually want to. were going to use this tool here called. super

In [209]:
print(predictions.iloc[88]['reason'])

The video snippet mentions both traditional and blockchain investments. The speaker discusses using an AI tool to create step-by-step guides for affiliate marketing, which is a traditional investment. However, the tool being used is an AI tool, which is related to blockchain investments. The speaker also mentions a keyword tool, which could be used for both traditional and blockchain investments. Overall, the video snippet is a mix of traditional and blockchain investments.


In [244]:
print(df.iloc[89]['messages'][3]['content'])

I Tried Selling AI Art For 30 Days. generator there are a bunch like Dolly 2. you generate unlimited images for free. with an Nvidia GPU lets generate some. good youre down to sell some air art. selling custom air on Fiverr it is a. custom art to get orders we need to. out what type of art well be selling. Ive just made five gigs on Fiverr. Landscape art heavy metal and rock album. covers anime and furry character art. Renaissance weve got all art styles. youre already thinking about starting. your own AI art hustle or any business. in general Ive put the template in the. starting a business right now download. Pet Portraits and concept art arent. doing too hot metal album covers are. album genre metal oh my God he wants a. metal album cover this was sent a week. customer come to us I think we need to. and Dragons art was one of our most. groups to promote it seems like one way. posts from people looking for an artist. its getting some likes is my art that. good people are commenting t

In [211]:
print(predictions.iloc[89]['reason'])

The video snippet discusses both traditional and blockchain investments. The speaker talks about generating AI art using an Nvidia GPU, selling custom art on Fiverr, and promoting the art on social media. They mention selling landscape art, heavy metal and rock album covers, anime and furry character art, and Renaissance art. They also talk about getting orders for pet portraits, concept art, and Dungeons and Dragons art. Additionally, they mention generating AI art for free and using it to start a business.


In [212]:
print(df.iloc[90]['snippet'])

The Secret to Making $250 per Day with Ai Art. actually generate money in this video.
technology to generate images like you.
actually ask the bot to create as the.
images and theres a lot of people.
prompt and ask it to create as an image.
image so you can start to give it a lot.
images that people are requesting look.
type something like abstract neon art.
actually choose one of these images for.
larger image of this abstract piece of.
image which appeals to a specific type.
paste in this image and we can use this.
want to create an image that might.
we can create an image that is kind of.
would like this image that is fantastic.
created its four Images which appeal to.
use to make money from these images the.
use to make money from these images the.
is we can actually just sell this image.
as a digital image on a website like.
digital print and people are actually.
selling and buying digital prints that.
images lets be honest I actually think.
Etsy account you can create a store and

In [213]:
print(predictions.iloc[90]['reason'])

The video snippet mentions the use of AI technology to generate images that can be sold as digital prints on websites like Etsy. The focus is on the use of AI in creating art and selling it as an investment.


In [245]:
print(df.iloc[91]['messages'][3]['content'])

Charlie Munger: Bitcoin Is Worthless Artificial Gold | CNBC. bitcoin is worthless artificial gold. theres a lot of illicit activity now. that is not something I think the world. computer science doesnt mean that it. speculate on Bitcoin reminds me of Oscar. he was before we asked earlier Charlie. though Andrew brought it up with Warren. but I think its a scumball activity. yeah is that better serve you better. thank you yeah we we asked earlier about. Goldman Sachs getting into the business. of having a trading desk for Bitcoin. Berkshire Hathaway owns about two and. half billion dollars of Goldman Sachs. surprise you just well I dont expect. every investment bank to agree with. everything I think theyre they have a. lot of animal spirits and investment. banking bill Charlie and Warren have. weighed in on Bitcoin do you own any. and then a few years later I thought hey. Im gonna sell that so no theres some. really good technology in terms of. blockchain that is a good thing Bitcoin. an

In [215]:
print(predictions.iloc[91]['reason'])

The snippet mentions Bitcoin, blockchain, and cryptocurrency, which are all related to blockchain investments. Charlie Munger's negative comments about Bitcoin also suggest that the snippet is related to blockchain investments.


In [216]:
print(df.iloc[92]['snippet'])

Charlie Munger: Bitcoin is Worthless Rat Poison. well bitcoin as a the computer science.
thats what captivates a lot of these.
people theyve actually created a.
product thats hard to create more of.
compute computer science people love it.
people are buying just because they.
has no intrinsic value and so I regard.
lot of money trading freshly harvested.
say thats immoral.
too awful a concept well to me bitcoin.
weve got debit cards isnt like we.
dont have a payment system weve got.
payment system China doesnt need.
Bitcoin and if you have a wee chat why.
right mind would want a payment system.
where the very thing you were using went.
no I regard the thing as a as a.
and I think the people that are pushing.
that ought to be some things that are.
is one we did not need a gold.
Bitcoin is worthless artificial gold.
had a lot of illicit activity now that.
is not something I think the world needs.
widely used in the respectable people.
should encourage other people to.
speculate on Bitcoi

In [217]:
print(predictions.iloc[92]['reason'])

The video snippet mentions Bitcoin, cryptocurrency, and blockchain, which are all related to Web3 investments. Charlie Munger's negative comments on Bitcoin and cryptocurrency suggest that the video is discussing blockchain investments.


In [ ]:
## I can argue this as a traditional video

In [248]:
print(df.iloc[93]['messages'][3]['content'])

Why Are Tech Stocks Taking Such a Beating? | WSJ. [Narrator] While other sectors have faced declines, heres why the tech industry is having a record downturn after years of growth. low interest rates, theyre good for tech stocks, lower yields make it really, really easy to, they increase the appeal of the future earnings that a lot of these tech stocks will have. [Narrator] In 2012 and 2013, tech players like Facebook and Twitter went public, (dramatic music) and advancements like mobile and cloud technology helped drive profits for many companies and fueled growth in the sector. By the end of that year, tech was at the forefront of the stock market, with a bull run being driven by companies like Apple and Microsoft. [Narrator] Stocks across the board initially took a plunge as "Stay at home" orders and lockdowns rippled across the world. [Narrator] As stimulus flooded the economy, market cap prices for tech leaders like Amazon, Apple, and Meta reached new records. Some companies signi

In [219]:
print(predictions.iloc[93]['reason'])

The video snippet discusses the stock market and the performance of tech companies, including Apple and Microsoft. It also mentions interest rates and their impact on tech stocks. The snippet does not mention any keywords related to blockchain or Web3 investments.


In [ ]:
## I can argue this as a traditional video

In [247]:
print(df.iloc[94]['messages'][3]['content'])

Why Wall Street Is Buying So Many U.S. Homes. Whats outrageous is your tax dollars are helping Wall Street buy up single family homes. This growing industry buys or builds single family homes and then rents them out. Rent hikes for single family homes in this region have outpaced other parts of the country since the onset of the pandemic. And theyre holding these houses often for years, taking them off the market, making it even harder for young people and working class families to own a house. 6 million single family rental homes. You started to see the emergence of professional management in the single family rental home space. They figured out pretty early on, like its pretty hard to be profitable when youre focusing on people who are missing payments all the time and living in 70 year old homes that need a lot of CapEx. So the Wall Street group has focused on newer homes and higher paying higher income tenants who are more discretionary renters, not not out of necessity because tha

In [221]:
print(predictions.iloc[94]['reason'])

This video snippet discusses Wall Street's growing industry of buying or building single-family homes and renting them out. It highlights the impact on rent hikes and the difficulty for young people and working-class families to own a house. The focus is on the profitability of newer homes and higher-paying tenants, which is a better profit model for Wall Street. The snippet also mentions the lack of meaningful investments in building new single-family homes.


In [246]:
print(df.iloc[95]['messages'][3]['content'])

The Craziest Wallstreetbets Trades You Never Heard About. of millions of them investing trading. basically makes two thousand dollars up. dollars in after hours trading the stock. forced to buy 23 million dollars worth. a 50 000 account holding 23 million. dollars worth of spy shares just to put. million worth of shares sold the. thousand dollars leaving him with a. great trades on the internet like i. thousands millions of dollars from. million dollar loss oh my god four. million dollars evaporated in one day. market volatility index if you look at. risk-free money right over a long enough. term it seems like volatility will. then basically one day the market sold. off like five percent causing the vix to. xiv to lose like 90 percent of its value. a conventional loss like this one as bad. when you lose 4 million dollars the. they know i really like it thats why. millions of dollars gone financial. fund and let your money multiply 22. gains are more complicated than losses. things are 

In [223]:
print(predictions.iloc[95]['reason'])

The video snippet discusses both traditional investments (e.g., stocks, trading, market volatility index) and blockchain investments (e.g., cryptocurrency options market). The snippet also mentions high-risk trades with potential for significant gains and losses, which could apply to both traditional and blockchain investments.


In [37]:
# Labels:

# Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
# Blockchain: Videos that recommend or educate about cryptocurrency (BTC, ETH, etc.), NFTs, or other Web3 investments or keywords related to them.
# Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
# Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

# Instructions:
# - The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
# - If talks about making money from jobs, side hustles, or other alternative assets (cars, watches, artificial intelligence, trading cards, art, etc), they are Unrelated.
# - A video that is downplaying an investment or discussing it negatively should be classified as Unrelated.
# - Please return predictions in the format" {Label} : {20-word or shorter rationale

In [252]:
completions = []
completion_df = []

pbar = tqdm(df.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    completion['message'] = entry['messages']
    completion['title'] = entry['title']
    completion['transcript'] = entry['cleaned_transcript']
    completion['snippet'] = entry['snippet']
    completion['link'] = entry['link']

    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(5)

Processing NFTs, Explained: : 100it [12:34,  7.54s/it]                                                                                    


In [7]:
predictions = pd.read_csv('seed_predictions.csv')

In [8]:
preds = list(predictions['prediction'].apply(lambda x: 'None' if x == 'Unrelated' else x))
actual = list(df['label'])
preds = ['Blockchain' if pred == 'Label' else pred for pred in preds]
sum(predictions['total_tokens']) * TURBO_COST

0.15497

In [10]:
from sklearn.metrics import accuracy_score, classification_report

In [11]:
accuracy_score(actual, preds)

0.88

In [12]:
print(classification_report(actual, preds))

              precision    recall  f1-score   support

  Blockchain       1.00      0.90      0.95        40
       Mixed       0.36      1.00      0.53         5
        None       0.89      0.53      0.67        15
 Traditional       0.95      0.97      0.96        40

    accuracy                           0.88       100
   macro avg       0.80      0.85      0.78       100
weighted avg       0.93      0.88      0.89       100



In [263]:
predictions

,role,content,prediction,reason,message,title,transcript,snippet,link,created,id,model,object,prompt_tokens,completion_tokens,total_tokens
0,assistant,Traditional: The video snippet discusses inves...,Traditional,The video snippet discusses investing in stock...,"[{'role': 'system', 'content': 'You are a clas...",How to Invest for Beginners,"Right, so let's say\nyou want to get started w...",How to Invest for Beginners. Maybe youve heard...,www.youtube.com/watch?v=gFQNPmLKj1k,1678267686,chatcmpl-6rkNaHwizA0nMF5QFF26RDUkUCkmn,gpt-3.5-turbo-0301,chat.completion,685,45,730
1,assistant,Traditional: The video snippet discusses inves...,Traditional,The video snippet discusses investing in tradi...,"[{'role': 'system', 'content': 'You are a clas...",I asked a personal finance expert how to invest.,"- Investing, when most people think of it, the...",I asked a personal finance expert how to inves...,www.youtube.com/watch?v=GZ42PIi9bis,1678267693,chatcmpl-6rkNhDSbB3VMpkEsDOd5r4wb3deK8,gpt-3.5-turbo-0301,chat.completion,684,50,734
2,assistant,Traditional : The video snippet provides an ex...,Traditional,The video snippet provides an explanation of t...,"[{'role': 'system', 'content': 'You are a clas...",Traditional and Roth IRAs | Simple Steps for a...,"If you're saving for retirement, IRAs, also\nk...",Traditional and Roth IRAs | Simple Steps for a...,www.youtube.com/watch?v=Z3I8TPYn4Z0,1678267700,chatcmpl-6rkNosTw0VxR2n4pQC5fWNQcQzDxT,gpt-3.5-turbo-0301,chat.completion,687,57,744
3,assistant,Traditional: The video snippet provides a deta...,Traditional,The video snippet provides a detailed explanat...,"[{'role': 'system', 'content': 'You are a clas...",What is a 401(k)?,One of the most common ways people save for\nr...,What is a 401(k)?. One of the most common ways...,www.youtube.com/watch?v=ss5yImerkhg,1678267708,chatcmpl-6rkNw9M2VAVErKjDcGM3BxxC9fUby,gpt-3.5-turbo-0301,chat.completion,713,73,786
4,assistant,Traditional: The video snippet talks about ret...,Traditional,The video snippet talks about retirement accou...,"[{'role': 'system', 'content': 'You are a clas...",How IRAs Work And Why They Are More Popular Th...,are you one of the 60 million americans. acti...,How IRAs Work And Why They Are More Popular Th...,www.youtube.com/watch?v=DPnXJpDOAHI,1678267716,chatcmpl-6rkO4PzLQpmFNeVi80zf4vpmVsu1z,gpt-3.5-turbo-0301,chat.completion,736,54,790
5,assistant,Traditional: The video snippet discusses index...,Traditional,"The video snippet discusses index funds, which...","[{'role': 'system', 'content': 'You are a clas...",What Are Index Funds?,"Funds that track a market index, such as the\n...",What Are Index Funds?. These funds typically u...,www.youtube.com/watch?v=3rFjS2yNjqc,1678267723,chatcmpl-6rkOBkuaS6GEbliDklJbb5KbSp93P,gpt-3.5-turbo-0301,chat.completion,733,66,799
6,assistant,Traditional: The video snippet discusses mutua...,Traditional,"The video snippet discusses mutual funds, port...","[{'role': 'system', 'content': 'You are a clas...",What The Heck Is An Index Fund?,"From an early age, we’re taught to celebrate\n...",What The Heck Is An Index Fund?. So how do you...,www.youtube.com/watch?v=u5eewhr7KKY,1678267730,chatcmpl-6rkOIWzIIGe9iTEitoAyC4rT3RhLz,gpt-3.5-turbo-0301,chat.completion,730,45,775
7,assistant,Traditional: This video snippet recommends and...,Traditional,This video snippet recommends and educates abo...,"[{'role': 'system', 'content': 'You are a clas...",How To Become A Millionaire: Index Fund Invest...,what's up you guys it's Graham here so. let's...,How To Become A Millionaire: Index Fund Invest...,www.youtube.com/watch?v=fwe-PjrX23o,1678267738,chatcmpl-6rkOQU9ewhzp2RAWtHUnTFYDXPA3A,gpt-3.5-turbo-0301,chat.completion,697,59,756
8,assistant,Traditional: The video snippet discusses the b...,Traditional,The video snippet discusses the basics of bond...,"[{'role': 'system', 'content': 'You are a clas...",Investing Basics: Bonds,"Bonds are a common investment. However, to man...",Investing Basics: Bonds. So lets e

In [19]:
predictions.iloc[45]['reason']
# Predicted mixed because the recommender was skeptical, thoiugh the title says coin will skyrocket, adjust prompt accordingly

"The snippet discusses both traditional investments (stocks) and blockchain investments (BTC, XRP, USDC, grayscale, finance, FTX, shiba, tether). The speaker mentions their success in getting people into Bitcoin when it was at a low price and emphasizes the importance of knowing when to sell. However, the snippet also includes criticism of the speaker's focus on crypto and skepticism about its long-term potential."

In [14]:
predictions.iloc[45]['snippet']

'Bitcoin and Altcoins Will Skyrocket After The Halving Event. so were talking about more than two years ago some YouTuber made a video about me I dont know his name I dont want to know his name May of 2020 so take a look the last one were going to talk about is clear value tax who seems pretty smart about this because he already probably analyzed the same information and realize that he does need to Pivot and build out a community elsewhere it looks like hes mostly focused on investing apps which I think makes sense as well as crypto which I think is a bit weird but well see what happens okay so he said that what Im doing with kryptos is weird but lets see what happens you know what happens I got people into Bitcoin at three thousand to five thousand dollars I made people a lot of money he made his video right about here which I circled in red and that was the price of Bitcoin at the time he said that was weird of me because I was talking about cryptos before Bitcoin was all the craze 

In [271]:
predictions.iloc[64]['reason']
# impact of social media led to this being predicted as mixed, also its sus about unrelated terminology like (like and subscribe)

'The video snippet talks about the rise of Dogecoin, which is a cryptocurrency (Blockchain). It also mentions the impact of social media and celebrity endorsements on the cryptocurrency market, which is related to traditional investments. However, it also includes language that is not related to either traditional or blockchain investments, such as "Smash like subscribe," "freaking out," and "love this channel."'

In [16]:
print(predictions.iloc[64]['snippet'])

The Rise of Dogecoin #doge #dogecoin. there guys and gals were going to be talking about the rise of douche coin what are we.
and guys were just gonna have a huge douche coin Jewish coin video guys so Doges going.
to the Moon dose gonna move dont get Smash like subscribe and remember none of this absolutely.
wider cryptocurrency market and its potential future guys this is going to be amazing this.
and share this video right freaking out guys lets get it lets do it dont go to the Moon.
as you guys all know initially Dogecoin was created as a joke guys Im talking a real.
well the role of social media and celebrity endorsements has played a huge role in Dogecoin.
and higher pop culture phenomenon as you guys know Dogecoin has become a pop cultural phenomena.
is fantastic it is cruel impact on the cryptocurrency market dogecoins surge in popular has had.
a significant impact on the wider cryptocurrency on markets you know we saw you know Dogecoin.
love the dose you love the Doge guys and 

In [272]:
predictions.iloc[65]['reason']
# classifying BTC as traditional, talks about side hustles and other information as well

'The snippet mentions both traditional investments (BTC, XRP) and blockchain investments (SHIBA INU, DOGE). However, the video also includes irrelevant information about side hustles and alternative assets, which makes it difficult to determine the main focus of the video.'

In [17]:
print(predictions.iloc[65]['snippet'])

SHIBA INU  ABOUT TO PUMP,  SHIBARUIM  ON SCHEDULE?  Why  I AM  BUYING More Shiba Inu! #DOGE #XRP. right now its you know its running about 25 000  so that is pretty peachy King guys so were gonna  .
be talking about uh some interesting stuff thats  going to be happening should you be buying should  .
the Nang Vietnam guys it is looking pretty good  guys so dont forget weve got I dont know if  .
holding holding their Bitcoin thats what that  means right now so lets talk about xrp uh for  .
going to be successful with their appending  court case thats going on right now you know  .
is going to be doing some making some movie move  move its coming pretty soon guys so uh you know  .
not liked xrp because of this pending uh court  case thats happening but uh you know all all  .
and you and YouTube black youre gonna love it and  you know whats going to happen is if more people  .
shirt right so thats one of the reasons why Im  a bit salty about xrp but Im going to be sweet  .
one now dont fo

In [23]:
# Labels:

# Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
# Blockchain: Videos that recommend or educate about cryptocurrency (BTC, ETH, etc.), NFTs, or other Web3 investments or keywords related to them.
# Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
# Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

# Instructions:
# - The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
# - If talks about making money from jobs, side hustles, or other alternative assets (cars, watches, artificial intelligence, trading cards, art, etc), they are Unrelated.
# - If a video is only discussing the asset negatively, it should be classified as Unrelated.
# - Please return predictions in the format" {Label} : {20 word or shorter rationale}

In [27]:
completions = []
completion_df = []

pbar = tqdm(df.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    completion['message'] = entry['messages']
    completion['title'] = entry['title']
    completion['transcript'] = entry['cleaned_transcript']
    completion['snippet'] = entry['snippet']
    completion['link'] = entry['link']

    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(5)

Processing NFTs, Explained: : 100it [11:13,  6.73s/it]                                                                                    


In [28]:
predictions = pd.read_csv('seed_predictions.csv')

In [29]:
preds = list(predictions['prediction'].apply(lambda x: 'None' if x == 'Unrelated' else x))
actual = list(df['label'])
preds = ['Blockchain' if pred == 'Label' else pred for pred in preds]
sum(predictions['total_tokens']) * TURBO_COST

0.151138

In [30]:
accuracy_score(actual, preds)

0.86

In [31]:
print(classification_report(actual, preds))

              precision    recall  f1-score   support

  Blockchain       0.95      0.90      0.92        40
       Mixed       0.33      1.00      0.50         5
        None       1.00      0.40      0.57        15
 Traditional       0.95      0.97      0.96        40

    accuracy                           0.86       100
   macro avg       0.81      0.82      0.74       100
weighted avg       0.93      0.86      0.87       100



In [36]:
predictions

,role,content,prediction,reason,message,title,transcript,snippet,link,created,id,model,object,prompt_tokens,completion_tokens,total_tokens
0,assistant,Traditional: The video discusses investing in ...,Traditional,"The video discusses investing in stocks, share...","[{'role': 'system', 'content': 'You are a clas...",How to Invest for Beginners,"Right, so let's say\nyou want to get started w...",How to Invest for Beginners. Maybe youve heard...,www.youtube.com/watch?v=gFQNPmLKj1k,1678309974,chatcmpl-6rvNefuqw7vh0HrEnBNWH8Xjcurvk,gpt-3.5-turbo-0301,chat.completion,684,47,731
1,assistant,Traditional: The video snippet discusses inves...,Traditional,The video snippet discusses investing in tradi...,"[{'role': 'system', 'content': 'You are a clas...",I asked a personal finance expert how to invest.,"- Investing, when most people think of it, the...",I asked a personal finance expert how to inves...,www.youtube.com/watch?v=GZ42PIi9bis,1678309981,chatcmpl-6rvNlALcdi5B6KP3ApOZqTA8ueTpn,gpt-3.5-turbo-0301,chat.completion,683,45,728
2,assistant,Traditional : The video snippet discusses trad...,Traditional,"The video snippet discusses traditional IRAs, ...","[{'role': 'system', 'content': 'You are a clas...",Traditional and Roth IRAs | Simple Steps for a...,"If you're saving for retirement, IRAs, also\nk...",Traditional and Roth IRAs | Simple Steps for a...,www.youtube.com/watch?v=Z3I8TPYn4Z0,1678309988,chatcmpl-6rvNsWLnH7APSoEshxn9ATRjnYgu4,gpt-3.5-turbo-0301,chat.completion,686,42,728
3,assistant,Traditional: The video snippet discusses a ret...,Traditional,The video snippet discusses a retirement savin...,"[{'role': 'system', 'content': 'You are a clas...",What is a 401(k)?,One of the most common ways people save for\nr...,What is a 401(k)?. One of the most common ways...,www.youtube.com/watch?v=ss5yImerkhg,1678309994,chatcmpl-6rvNye36DnIZFQ6Lw9MUE4d6Sjyli,gpt-3.5-turbo-0301,chat.completion,712,43,755
4,assistant,Traditional: The video snippet discusses retir...,Traditional,The video snippet discusses retirement account...,"[{'role': 'system', 'content': 'You are a clas...",How IRAs Work And Why They Are More Popular Th...,are you one of the 60 million americans. acti...,How IRAs Work And Why They Are More Popular Th...,www.youtube.com/watch?v=DPnXJpDOAHI,1678310001,chatcmpl-6rvO5NpcSTL3xx6qlGAZPcTu6eahm,gpt-3.5-turbo-0301,chat.completion,735,42,777
5,assistant,Traditional: The video snippet discusses index...,Traditional,"The video snippet discusses index funds, which...","[{'role': 'system', 'content': 'You are a clas...",What Are Index Funds?,"Funds that track a market index, such as the\n...",What Are Index Funds?. These funds typically u...,www.youtube.com/watch?v=3rFjS2yNjqc,1678310008,chatcmpl-6rvOCVxXfVr15D1G4wSroZSyKUaO6,gpt-3.5-turbo-0301,chat.completion,732,67,799
6,assistant,Traditional: The video snippet discusses mutua...,Traditional,"The video snippet discusses mutual funds, port...","[{'role': 'system', 'content': 'You are a clas...",What The Heck Is An Index Fund?,"From an early age, we’re taught to celebrate\n...",What The Heck Is An Index Fund?. So how do you...,www.youtube.com/watch?v=u5eewhr7KKY,1678310015,chatcmpl-6rvOJ5usInB5NGfucMsO9QA62yVQl,gpt-3.5-turbo-0301,chat.completion,729,46,775
7,assistant,Traditional: This video snippet discusses inve...,Traditional,This video snippet discusses investing in inde...,"[{'role': 'system', 'content': 'You are a clas...",How To Become A Millionaire: Index Fund Invest...,what's up you guys it's Graham here so. let's...,How To Become A Millionaire: Index Fund Invest...,www.youtube.com/watch?v=fwe-PjrX23o,1678310022,chatcmpl-6rvOQibpY1f9GJhpF3isXOujUlJ17,gpt-3.5-turbo-0301,chat.completion,696,41,737
8,assistant,Traditional: The video snippet discusses the b...,Traditional,The video snippet discusses the basics of bond...,"[{'role': 'system', 'content': 'You are a clas...",Investing Basics: Bonds,"Bonds are a common investment. However, to man...",Investing Basics: Bonds. So le

In [41]:
print(predictions.iloc[0]['snippet'])

How to Invest for Beginners. Maybe youve heard on the news about Tesla or Netflix or Amazon and how, if youd invested 10 years ago in Tesla then youd be a millionaire by now or things like that.
It is the video I wish I would have had five years ago when I first started investing in stocks and shares.
And if we adjust for inflation that our money is still worth 206 pounds in 10 years time, this is pretty good.
So 1% of a million pounds, which is 10,000 pounds obviously no one watching this actually owns 1% of Apple, unless Tim Cook, youre watching, I dont even know if you own that much because that would make you an extremely rich person because Apple is a very valuable company but thats basically how the dividend thing works.
The second way of making money from shares is sort of like with houses in that you get the capital gains over time.
And at this point youve probably got a few questions like how much money you need to get started or how risky is buying shares in a company.
Im gon